In [5]:
import pandas as pd
import numpy as np
!pip install networkx
import networkx as nx
import re
import scipy.spatial.distance as sp

!pip install gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import string

### 1.Define Model \ Doc2Vec

In [59]:
# DOC2VEC

tokenized_doc = []
for d in sentences:
    tokenized_doc.append(word_tokenize(d.lower()))


tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]

## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)
# Save trained doc2vec model
model.save("test_doc2vec.model")
## Load saved doc2vec model
model= Doc2Vec.load("test_doc2vec.model")
## Print model vocabulary
model.wv

for sentence in sentences:
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
   
    words = sentence.split(" ")
    v = [word for word in words]
    [model.wv.n_similarity(v1,v2)  for v1, v2 in zip(v,v[1:])]

#### 1.1. Get Sentences Into DF

In [68]:
list_sentences = ["Under the Biden's presidency, the gas prices have decreased to their pre-Russian invasion of Ukraine levels.",
"Wisconsin's archaic abortion ban is older than 20 states.",
"The USA has a gun homicide rate 26 times higher than our peers.",
"86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check.",
"It's just crazy': 12 major cities hit all-time homicide records",
"Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one.",
"When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it.",
"Each of the pro-life governors up for election won in November.",
"The Trump administration worked to free 5,000 prisoners of Taliban.",
"Biden reduced the Strategic Petroleum Reserves of the United States to their lowest level since 1984. The reserve used to had more than 600 million barrels.",
"GOP senators introduce bill designating Mexican drug cartels as terror organizations",
"The inflation is costing American families more than $250 per month under Biden's policies.",
"Donald Trump and his campaign donated food and supplies to Hurricane Matthew victims",
"The average family in America pays more money for to their local hospital than to the IRS.",
"Joe Biden aggrees transgender women aren't real women! He told a transgender women you will never be a realm woman.",
"A photo shows Barack Obama dressed up as Baphomet (Lucifer).",
"Biden didn’t win in Arizona. He lost in Arizona based on the forensic audit.",
"It is known that no candidate has ever won both Florida and Ohio and lost, but Trump did.",
"There are 63 million abortions a year in this country.",
"A new study finds only 6% of the people actually died from COVID. The others died from other reasons.",
"Video shows Joe Biden putting a Medal of Honor on a Vietnam War veteran backward.",
"83 percent of the benefits from Republican tax reform in 2017 went to the wealthiest 1 percent.",
"There is an increase in Social Security checks for seniors for the first time in 10 years under president Biden.",
"Donald Trump has not apologized for comments caught on tape by Access Hollywood.",
"When Trump won, we had the lowest voter turnout in 20 years.",
"US spend twice as much per capita on health care as any other nation on Earth.",
"Last year, Biden said they funded 700,000 major construction projects.",
"Trump’s tax plan was completely focused on the wealthy and the powerful - not the middle class."]

In [69]:
df = pd.DataFrame(list_sentences)

In [70]:
df.columns=['text']

### 1.2. Get Similarities

#### 1.2.1. Make them Dictionary

In [153]:
mydict={}
for i,sentence in enumerate(list_sentences):
    try:
        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        words = sentence.split(" ")
        similarities = [model.wv.n_similarity(v1, v2) for v1, v2 in zip(words, words[1:])]
        pairs = [(v1, v2) for v1, v2 in zip(words, words[1:])]      
    except:
        pass
    for a in range(len(similarities)):
        " Get them into dictionary sentence no: similarities"
        mydict[i]=similarities
print(mydict)                   

{0: [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555], 1: [0.33759552, 0.882059, 0.882059, 0.33759555, 0.0, 0.0, 0.0, 0.0], 2: [0.4849517, 1.0, 0.84166795, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 3: [0.0, 0.0, 0.9401378, 0.0, 0.0, 0.0, 0.0, 0.7848633, 0.0, 0.0, 0.0, 0.0, 0.26141003, 0.42438176, 0.69202626, 1.0, 0.0], 4: [0.99999994, 0.5401927, 0.0, 0.0, 0.5401927, 0.8032929, 0.8592872, 0.0, 0.0], 5: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7649056, 0.8032929, 0.5169574, 0.5169574, 0.7533626, 0.88003224, 0.84166795, 0.33759555, 0.0], 6: [0.0, 0.0, 0.0, 0.0, 0.98015374, 0.7776745, 1.0, 0.5169574, 0.882059, 0.0, 0.0, 0.0, 0.0, 0.733976, 0.67274845, 0.0, 0.0, 0.0, 0.0], 7: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 8: [1.0, 0.8294905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 9: [0.0, 0.0, 0.8868668, 0.8868668, 0.26141003, 0.0, 0.0, 1.0, 0.80353093, 0.80353093, 1.0, 0.8032929, 0.0, 0.0, 

In [158]:
for key in mydict:
    print(mydict[key])

[0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
[0.33759552, 0.882059, 0.882059, 0.33759555, 0.0, 0.0, 0.0, 0.0]
[0.4849517, 1.0, 0.84166795, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.9401378, 0.0, 0.0, 0.0, 0.0, 0.7848633, 0.0, 0.0, 0.0, 0.0, 0.26141003, 0.42438176, 0.69202626, 1.0, 0.0]
[0.99999994, 0.5401927, 0.0, 0.0, 0.5401927, 0.8032929, 0.8592872, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7649056, 0.8032929, 0.5169574, 0.5169574, 0.7533626, 0.88003224, 0.84166795, 0.33759555, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.98015374, 0.7776745, 1.0, 0.5169574, 0.882059, 0.0, 0.0, 0.0, 0.0, 0.733976, 0.67274845, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.8294905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.8868668, 0.8868668, 0.26141003, 0.0, 0.0, 1.0, 0.80353093, 0.80353093, 1.0, 0.8032929, 0.0, 0.0, 0.0, 0.0, 0.26141003, 1.0, 0.26141003, 0

In [99]:
for key in mydict:
     print(len(mydict[key]))

15
8
12
17
9
17
19
10
9
25
10
13
12
16
19
9
13
17
9
18
14
16
19
12
11
15
9
9


#### 2.3. Calculate Similarity

In [93]:
doc2vec_sims =[]
for key in mydict:
    doc2vec_sim = (np.mean(mydict[key]))
    doc2vec_sims.append(doc2vec_sim)

In [95]:
df['doc2vec_sims'] = doc2vec_sims

### 3. Save the Similarity Table

In [97]:
df.to_excel('similarities.xlsx')

In [67]:
for i,sentence in enumerate(df.text):
    try:
        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        words = sentence.split(" ")
        similarities = [model.wv.n_similarity(v1, v2) for v1, v2 in zip(words, words[1:])]
        pairs = [(v1, v2) for v1, v2 in zip(words, words[1:])]      
    except:
        pass
    for a in range(len(similarities)):
            print(i,similarities)

0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.0, 0.33759555]
0 [0.0, 0.26141003, 1.0, 0.26141003, 0.4849517, 0.7924363, 0.33759555, 0.84166795, 0.4849517, 1.0, 0.42438173, 0.97226405, 0.0, 0.

### Doc2vec

In [62]:
data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

max_epochs = 100
vector_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vector_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/Users/eotenen/mambaforge/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
/Users/eotenen/mambaforge/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).


In [23]:
for sentence in sentences:
    words = sentence.split(" ")
    words = [word.lower() for word in words]
    try:
        v = [model[word] for word in words]
        vd = [sp.euclidean(v1,v2) for v1, v2 in zip(v,v[1:])]
        print(sentence, np.std(vd))
    except:
        KeyError

In [6]:
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it",
            "Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one"]

 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

def preprocess(text):
    # Define the text to preprocess
    # Convert the text to lowercase
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Print the final list of preprocessed and tokenized words
    return(lemmatized_tokens)


In [8]:
for line in d:
        l = line.strip('\n')
        vals = l.split(" ")
        term2vec[vals[0]] = np.asfarray(vals[1:])

sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it",
            "Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one"]
             
for sentence in sentences:
    words = sentence.split(" ")
    words = [word.lower() for word in words]
    v = [term2vec[word] for word in words]
    vd = [sp.euclidean(v1,v2) for v1, v2 in zip(v,v[1:])]
    print(sentence, np.std(vd))


KeyError: '1970,'

In [38]:
glove_vectors = gensim.downloader.load( 'word2vec-google-news-300')

In [40]:
all_vectors = []
for index, vector in enumerate(model.wv.vectors):
    vector_object = {}

    vector_object[list(model.wv.key_to_index())[index]] = vector
    all_vectors.append(vector_object)

TypeError: 'dict' object is not callable

In [ ]:
from gensim.models import KeyedVectors
# Load vectors directly from the file

# Access vectors for specific words with a keyed lookup:
vector = glove_vectors['easy']
# see the shape of the vector (300,)
vector.shape
# Processing sentences is not as simple as with Spacy:
vectors = [glove_vectors[x] for x in "new york senate is insane".split(' ')]

### Term2vec

In [63]:
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it",
            "Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one"]
             

vectors = []
for sentence in sentences:
    words = sentence.split(" ")
    words = [word.lower() for word in words]
    try:
        vector = [glove_vectors[x] for x in words]
        vectors.append(vector)
    except:
        print(f'Error in finding token')

Error in finding token
Error in finding token
Error in finding token
Error in finding token
Error in finding token
Error in finding token
Error in finding token


In [ ]:
for line in d:
    l = line.strip('\n')
    vals = l.split(" ")
    term2vec[vals[0]] = np.asfarray(vals[1:])

    
for sentence in sentences:
    words = sentence.split(" ")
    v = [term2vec[word] for word in words]
    vd = [sp.euclidean(v1,v2) for v1, v2 in zip(v,v[1:])]
    print(sentence, np.std(vd))


In [ ]:
for idx, row in df.iterrows():
    for paragraph in (row['clean_text'].split(' ')):
        v = (term2vec[word] for word in paragraph)
        print(v)

### S-BERT

In [46]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it"]


def sim_calculator(sentences):
    similarities = []
    embeddings = []

    for sentence in sentences:
        words = sentence.split()
        words = [word.lower() for word in words]
        #print(words)

        embedding = model.encode(words)
        embeddings.append(embedding)
    return embeddings
    


/Users/eotenen/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
embeddings = sim_calculator(sentences)

In [48]:
len(embeddings[0])

20

In [49]:
#TRYING AND HERE!

In [50]:
[util.cos_sim(e1,e2) for e1, e2 in zip(embeddings[0][0],embeddings[0][1:])]



NameError: name 'util' is not defined

In [ ]:
len(embeddings)
sim = [util.cos_sim(e1,e2) for e1, e2 in zip(embeddings,embeddings[1:])]
mean = torch.mean(torch.stack(sim))
mean

In [51]:
import statistics
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it",
            "Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one"]
  
my = ['my cat drink milk',
     'i jump to dog to paint scream']

sim_calculator(my)
    

[array([[-0.02438809,  0.00035644,  0.02360098, ...,  0.03700326,
         -0.06691141,  0.01926723],
        [ 0.03733037,  0.0511618 , -0.00030604, ...,  0.06126916,
          0.06081468,  0.04928033],
        [ 0.00076719,  0.01072861,  0.01950581, ..., -0.01082846,
          0.06935582, -0.01736863],
        [-0.02547085, -0.02695013, -0.00880181, ..., -0.02022799,
          0.1713459 , -0.03877466]], dtype=float32),
 array([[-0.07408568,  0.00926614,  0.03269223, ...,  0.02666071,
          0.01889951, -0.08238821],
        [ 0.00781205,  0.00412357,  0.00737377, ..., -0.03441425,
         -0.06103116,  0.048816  ],
        [-0.02195791,  0.04292493, -0.04130689, ...,  0.12667814,
          0.04988158,  0.01570064],
        ...,
        [-0.02195791,  0.04292493, -0.04130689, ...,  0.12667814,
          0.04988158,  0.01570064],
        [-0.02627855,  0.08373255,  0.05348607, ...,  0.04548851,
         -0.01260481,  0.00908799],
        [ 0.01683358,  0.03489308, -0.01980388, ...,

In [52]:
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it"]


for sentence in sentences:
    # Tokenize the sentence into a list of words
    words = sentence.split()
    words = [word.lower() for word in words]
    
    #Sentences are encoded by calling model.encode()
    embeddings = model.encode(words)

#Print the embeddings
for words, embedding in zip(words, embeddings):
    #print("Words:", words)
    #print("Embedding:", embedding)

    vd = [sp.euclidean(v1,v2) for v1, v2 in zip(embedding,embedding[1:])]
    print(vd)


ValueError: Input vector should be 1-D.

In [44]:
#model = gensim.downloader.load('word2vec-google-news-300')


In [53]:
#from gensim.models import Word2Vec
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Load pre-trained Word2Vec model
# Example sentence
sentences = ["When the New York State Senate voted to legalize abortion in 1970, 12 Republican senators voted in favor of it",
            "Gas prices have decreased to their pre-Russian invasion of Ukraine levels",
             "Wisconsin's archaic abortion ban is older than 20 states",
             "The USA has a gun homicide rate 26 times higher than our peers.",
             "86% of Americans and 82% of gun owners support requiring all gun buyers to pass a background check",
             "Joe Biden’s $36 billion for a union pension fund is the largest private pension bailout in American history",
             "Biden says he released 22 years of my tax returns but Donald Trump hasn’t released a single one"]

for sentence in sentences:
    # Tokenize the sentence into a list of words
    words = sentence.split()
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    # Calculate similarities between all pairs of words
    try:
        similarities = []
        sim_num = []
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                similarity = model.similarity(words[i], words[j])
                similarities.append((words[i], words[j], similarity))
                sim_num.append(similarity)
                for similarity in similarities:
                    print(similarities)

    except:
        pass
    # Print the similarity scores


In [54]:
#from gensim.models import Word2Vec
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Load pre-trained Word2Vec model
# Example sentence
def similarity_calculation(sentences):
    for words in sentences:
        # Tokenize the sentence into a list of words
        words = words.split()
        words = [word.lower() for word in words]
        words = [word for word in words if word not in stop_words]
        # Calculate similarities between all pairs of words
        try:
            similarities = []
            sim_num = []
            for i in range(len(words)):
                for j in range(i+1, len(words)):
                    similarity = model.similarity(words[i], words[j])
                    similarities.append((words[i], words[j], similarity))
                    sim_num.append(similarity)
                    for similarity in similarities:
                        print(similarity)

        except:
            pass
    return (sim_num)


In [55]:
model.similarity('ege','deniz')

AttributeError: 'SentenceTransformer' object has no attribute 'similarity'

In [56]:
similarities_list=[]

for index, sentence in enumerate(df.text):
    # Tokenize the sentence into a list of words
    words = sentence.split()
    words = [word.lower() for word in words]
    try:
        similarities = []
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                similarity = model.similarity(words[i], words[j])
                similarities.append((words[i], words[j], similarity))

    except:
        pass
        print(similarities)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [57]:
similarities_list = []
for index, sentence in enumerate(df.text):
    # Tokenize the sentence into a list of words
    words = sentence.split()
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    # Calculate similarities between all pairs of words
    try:
        similarities = []
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                similarity = model.similarity(words[i], words[j])
                similarities.append((words[i], words[j], similarity))
                similarities_list.append(similarities)

    except:
        pass
    
print(similarities_list)

[]


In [58]:
for index, sentence in enumerate(df.text):
    # Tokenize the sentence into a list of words
    words = sentence[1].split()
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    # Calculate similarities between all pairs of words
    try:
        similarities = []
        sim_num = []
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                similarity = model.similarity(words[i], words[j])
                similarities.append((words[i], words[j], similarity))
                sim_num.append(similarity)
                for similarity in similarities:
                    print(similarity)
                    
        my_dictionary = {}
        my_dictionary[index] = sim_num
    except:
        pass


    print(my_dictionary)
    
## I could not store them

{0: []}
{1: []}
{2: []}
{3: []}
{4: []}
{5: []}
{6: []}
{7: []}
{8: []}
{9: []}
{10: []}
{11: []}
{12: []}
{13: []}
{14: []}
{15: []}
{16: []}
{17: []}
{18: []}
{19: []}
{20: []}
{21: []}
{22: []}
{23: []}
{24: []}
{25: []}
{26: []}
{27: []}
